<a href="https://colab.research.google.com/github/paulalias452/Deep-Learning-Assignments/blob/main/GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GAN Pracs

6/8/22

In [ ]:
import numpy as np
import pandas as pd
import keras
from keras.layers import Conv2D, Conv2DTranspose, Dropout, Input, Flatten, LeakyReLU, Dense, Reshape
from keras.models import Model, Sequential
from keras.datasets import mnist
from keras.layers.advanced_activations import LeakyReLU

In [ ]:
from keras.utils.vis_utils import plot_model
from numpy import vstack, ones
from numpy.random.mtrand import rand, randint, randn
(trainX, trainy),(testX,testy) =mnist.load_data()

print('Train', trainX.shape,trainy.shape)
print('Test', testX.shape,testy.shape)

Train (60000, 28, 28) (60000,)
Test (10000, 28, 28) (10000,)


In [ ]:
def define_discriminator(in_shape=(28,28,1)):
  model=Sequential()
  model.add(Conv2D(64,(3,3),strides=(2,2),padding='same',input_shape=in_shape))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.4))
  model.add(Conv2D(64,(3,3),strides=(2,2),padding='same'))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Dropout(0.4))
  model.add(Flatten())
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model


In [ ]:
d_model=define_discriminator()

def load_real_samples():
  X=np.expand_dims(trainX, axis=-1)
  X=X.astype('float32')
  X=X/255.0
  return X

In [ ]:
def generate_real_samples(dataset, n_samples):
  ix=randint(0, dataset.shape[0],n_samples)
  #retreieve selected images
  X=dataset[ix]
  #generate 'real' class labels
  y=ones((n_samples,1))
  return X,y

In [ ]:
def generate_fake_samples(n_samples):
  #generate uniform random number in [0,1]
  X=rand(28 * 28 * n_samples)
  X=X.reshape((n_samples,28, 28, 1))
  #generate fake class labels (0)
  y=np.zeros((n_samples, 1))
  return X, y

In [ ]:
def train_discriminator(model, dataset, n_iter=100, n_batch=256):
  half_batch=int(n_batch/2)
  for i in range(n_iter):
    X_real, y_real=generate_real_samples(dataset, half_batch)
    _,real_acc=model.train_on_batch(X_real, y_real)
    X_fake,y_fake=generate_fake_samples(half_batch)
    _, fake_acc= model.train_on_batch(X_fake,y_fake)
    print('>%d real=%.0f%% fake=%.0f%%' % (i+1, real_acc *100,fake_acc*100))

model= define_discriminator()
model.summary()
#plot_model(model, to_file='discriminator_plot.png', show_shapes=True, show_layer_names=)
dataset=load_real_samples()
train_discriminator(model, dataset)

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 14, 14, 64)        640       
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 14, 14, 64)        0         
                                                                 
 dropout_12 (Dropout)        (None, 14, 14, 64)        0         
                                                                 
 conv2d_13 (Conv2D)          (None, 7, 7, 64)          36928     
                                                                 
 leaky_re_lu_13 (LeakyReLU)  (None, 7, 7, 64)          0         
                                                                 
 dropout_13 (Dropout)        (None, 7, 7, 64)          0         
                                                                 
 flatten_6 (Flatten)         (None, 3136)             